In [ ]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [ ]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
nan_list = [None, 'None', '', ' ', [], -1, '-1', 0, '0', 'nan', np.nan, 'Nan']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'


In [ ]:
import string
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import numpy as np
import googletrans
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [ ]:
# This is a manually collected dictionary of incorrect/faulty keywords in scraped site data
with open(f'{scraped_data}CBS/Data/keyword_trans_dict.txt') as f:
    keyword_trans_dict = json.load(f)


In [ ]:
# 111 words to fix
len(keyword_trans_dict)


In [ ]:
def fix_broken_linkedin_files(glob_path):
    fix_list = []
    data_dict = {}
    data_list = []

    if glob_path.endswith('.json'):

        with open(glob_path, encoding = 'utf-8') as csv_file_handler:
            csv_reader = csv.DictReader(csv_file_handler)

            for rows in csv_reader:
                first_key = str(list(rows.keys())[0])
                key = rows[first_key]
                data_dict[key] = rows

        for num in data_dict:
            data_list.append(data_dict[num])

        with open(glob_path, 'w', encoding = 'utf-8') as json_file_handler:
            json_file_handler.write(json.dumps(data_list, indent = 4))
    
    return data_list


In [ ]:
def fix_keywords(df_temp):
    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        for key, value in keyword_trans_dict.items():
            df_temp.loc[
                df_temp[df_temp['Search Keyword'].notnull()]['Search Keyword'].astype(str).apply(
                lambda x: x.lower().strip()
                ) == str(key).lower().strip(), 'Search Keyword'
            ] = str(value).lower().strip()

        unfixed = df_temp.loc[
            df_temp[df_temp['Search Keyword'].notnull()]['Search Keyword'].astype(str).apply(lambda x: x.lower().strip()).isin([x.lower().strip() for x in list(keyword_trans_dict.keys())])
        ]

        if len(unfixed) != 0:
            for key, value in keyword_trans_dict.items():
                for idx, row in df_temp.iterrows():
                    if row['Search Keyword'].astype(str).lower().strip() == str(key).lower().strip():
                        df_temp.loc[idx, 'Search Keyword'] = str(value).lower().strip()
    

    return df_temp


#### Read paths

In [ ]:
glob_paths = []

for site in site_list:
    glob_paths.extend(glob.glob(f'{scraped_data}/{site}/Data/*.json')+glob.glob(f'{scraped_data}/{site}/Data/*.csv'))


In [ ]:
# 955 json and csv files
len(glob_paths)


#### Use paths to open files, fix keywords, and drop unneeded columns

In [ ]:
# Fix list catches all incorrect/faculty keyword search terms

fix_list = []

# Appended data catches all the fixed and cleaned dfs
appended_data = []

for glob_path in glob_paths:

    if glob_path.endswith('.json'):
        try:
            df_temp = pd.read_json(glob_path).reset_index(drop=True)
        except ValueError:
            fix_list.append(glob_path)
            if 'scraped_data/LinkedIn/Data/linkedin_jobs_df_' in glob_path:
                data_json = fix_broken_linkedin_files(glob_path)
                try:
                    df_temp = pd.read_json(glob_path).reset_index(drop=True)
                except ValueError:
                    fix_list.append(glob_path)
    elif glob_path.endswith('.csv'):
        df_temp = pd.read_csv(glob_path).reset_index(drop=True)

    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        df_temp = fix_keywords(df_temp)
        df_temp.reset_index(drop=True, inplace=True)
        df_temp.drop(columns=cols, axis='columns', inplace=True, errors='ignore')
        df_temp.drop(
        df_temp.columns[
                df_temp.columns.str.contains(
                    'unnamed|index|level', regex=True, case=False, flags=re.I
                )
            ],
            axis='columns',
            inplace=True,
            errors='ignore',
        )
    
        if glob_path.endswith('.json'):
            df_temp.to_json(glob_path, orient='records')
        elif glob_path.endswith('.csv'):
            df_temp.to_csv(glob_path, index=False)

        appended_data.append(df_temp.reset_index(drop=True))

# Concatonate list of dfs into one large df_jobs
df_jobs = pd.concat(appended_data).reset_index(drop=True)

# Save df_jobs to file
if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw.pkl')
    
    df_jobs.to_csv(f'{data_dir}df_jobs_raw.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


In [ ]:
# List of dfs, len = 527
len(appended_data)


In [ ]:
# len = 204394
len(df_jobs)


In [ ]:
# If we couldn't fix some keywords, we add them to list fix_list and write to file
if len(fix_list) != 0:
    print('Some keywords to fix!')
    with open(f'{data_dir}fix_list.txt', 'w') as f:
        json.dump(fix_list, f)


### START HERE IF SOURCING FROM DF_RAW
### PLEASE SET CORRECT DIRECTORY PATHS BELOW

In [ ]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [ ]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
nan_list = [None, 'None', '', ' ', [], -1, '-1', 0, '0', 'nan', np.nan, 'Nan']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'


In [ ]:
import string
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import numpy as np
import googletrans
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [ ]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw.pkl').reset_index(drop=True)


In [ ]:
# len = 204394
len(df_jobs)


In [ ]:
df_jobs.info()


In [ ]:
df_jobs.columns

In [ ]:
# Clean columns
df_jobs.columns = df_jobs.columns.to_series().apply(lambda x: str(x).strip())


In [ ]:
# Drop NA
df_jobs.dropna(axis='index', how='all', inplace=True)
df_jobs.dropna(axis='columns', how='all', inplace=True)


In [ ]:
# len = 204394
len(df_jobs)


In [ ]:
# Drop duplicates on subset of 'Job Description'
df_jobs.drop_duplicates(subset=['Job Description'], keep='first', ignore_index=True, inplace=True)


In [ ]:
# len = 62579
len(df_jobs)


In [ ]:
# Remove any rows with missing 'Job Description'
df_jobs.drop(
    df_jobs.index[
        df_jobs['Job Description'].isin(nan_list) | 
        df_jobs.loc[df_jobs['Job Description'].isnull(), 'Job Description'] | 
        df_jobs.loc[df_jobs['Job Description'].isna(), 'Job Description']
    ], 
    axis='index',
    inplace=True,
    errors='ignore'

)


In [ ]:
# len = 62577
len(df_jobs)


In [ ]:
# Save df_jobs to file
if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw_dropped.pkl')
    
    df_jobs.to_csv(f'{data_dir}df_jobs_raw_dropped.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


### START HERE IF SOURCING FROM DF_RAW_DROPPED
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [1]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [2]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
nan_list = [None, 'None', '', ' ', [], -1, '-1', 0, '0', 'nan', np.nan, 'Nan']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'


In [3]:
import string
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import numpy as np
import googletrans
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [4]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_dropped.pkl').reset_index(drop=True)


In [5]:
# 62577
len(df_jobs)


62577

In [6]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62577 entries, 0 to 62576
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Search Keyword     62577 non-null  object 
 1   Platform           62577 non-null  object 
 2   Job ID             62577 non-null  object 
 3   Job Title          62577 non-null  object 
 4   Company Name       62574 non-null  object 
 5   Location           62577 non-null  object 
 6   Job Description    62577 non-null  object 
 7   Rating             3975 non-null   float64
 8   Employment Type    61995 non-null  object 
 9   Company URL        59263 non-null  object 
 10  Job URL            62577 non-null  object 
 11  Job Age            62577 non-null  object 
 12  Job Age Number     62577 non-null  object 
 13  Collection Date    62577 non-null  object 
 14  Data Row           58599 non-null  float64
 15  Tracking ID        58599 non-null  object 
 16  Industry           591

In [ ]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl').reset_index(drop=True)


In [ ]:
# # 64684
# len(df_jobs)


In [ ]:
# df_jobs['Language'].value_counts()

In [7]:
# def language_detection(idx, row, df_jobs):

#     translator = Translator()
#     googletrans_readtime_error = googletrans.client.httpx._client.httpcore._exceptions.ReadTimeout

#     try:
#         print(f'Row {idx}: Translation in progress.')
#         time.sleep(60)
#         df_jobs.loc[idx, 'Language'] = str(translator.detect(str(row['Job Description']).lower().strip()).lang)
#         print(f'Row {idx}: Translation done.')
#     except:# googletrans_readtime_error:
#         print(f'Row {idx}: Sleeping for an hour.')
#         print('-'*30)
#         time.sleep(3600)
#         print(f'Row {idx}: Done sleeping.')
#         print('-'*30)
#         df_jobs.loc[idx, 'Language'] = str(translator.detect(str(row['Job Description']).lower().strip()).lang)

#     return df_jobs

In [ ]:
translator = Translator()
googletrans_readtime_error = googletrans.client.httpx._client.httpcore._exceptions.ReadTimeout

if 'Language' not in df_jobs.columns:
    df_jobs['Language'] = np.nan # create Language col and fill it with nan

# try:
#     time.sleep(60)
#     df_jobs['Language'] = df_jobs['Job Description'].apply(lambda x: translator.detect(str(x).lower().strip()).lang)
# except:
#     time.sleep(3600)
#     df_jobs['Language'] = df_jobs['Job Description'].apply(lambda x: translator.detect(str(x).lower().strip()).lang)

for idx, row in df_jobs.iterrows():
    # This part ensures we don't start lang detection from index 0 if some lang detection was already done
    if len(str(row['Job Description'])) != 0:
        if type(row['Language']) == float and np.isnan(row['Language']): #if lang is nan, detect language
            
            try:
                print(f'Row {idx}: Translation in progress.')
                time.sleep(60)
                df_jobs.loc[idx, 'Language'] = str(translator.detect(str(row['Job Description']).lower().strip()).lang)
                print(f'Row {idx}: Translation done.')
            except:# googletrans_readtime_error:
                print(f'Row {idx}: Sleeping for an hour.')
                print('-'*30)
                time.sleep(3600)
                print(f'Row {idx}: Done sleeping.')
                print('-'*30)
                df_jobs.loc[idx, 'Language'] = str(translator.detect(str(row['Job Description']).lower().strip()).lang)

        else: # elif lang is not nan, skip and go to next idx
            continue

if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl')
    
    df_jobs.to_csv(f'{data_dir}df_jobs_raw_language_detected.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


Row 1: Translation in progress.
Row 1: Translation done.
Row 2: Translation in progress.
Row 2: Translation done.
Row 3: Translation in progress.
Row 3: Translation done.
Row 4: Translation in progress.
Row 4: Translation done.
Row 5: Translation in progress.
Row 5: Translation done.
Row 6: Translation in progress.
Row 6: Translation done.
Row 7: Translation in progress.
Row 7: Translation done.
Row 8: Translation in progress.
Row 8: Translation done.
Row 9: Translation in progress.
Row 9: Translation done.
Row 10: Translation in progress.
Row 10: Translation done.
Row 11: Translation in progress.
Row 11: Translation done.
Row 12: Translation in progress.
Row 12: Translation done.
Row 13: Translation in progress.
Row 13: Translation done.
Row 14: Translation in progress.
Row 14: Translation done.
Row 15: Translation in progress.
Row 15: Translation done.
Row 16: Translation in progress.
Row 16: Translation done.
Row 17: Translation in progress.
Row 17: Translation done.
Row 18: Transla

Row 139: Translation done.
Row 140: Translation in progress.
Row 140: Translation done.
Row 141: Translation in progress.
Row 141: Translation done.
Row 142: Translation in progress.
Row 142: Translation done.
Row 143: Translation in progress.
Row 143: Translation done.
Row 144: Translation in progress.
Row 144: Translation done.
Row 145: Translation in progress.
Row 145: Translation done.
Row 146: Translation in progress.
Row 146: Translation done.
Row 147: Translation in progress.
Row 147: Translation done.
Row 148: Translation in progress.
Row 148: Translation done.
Row 149: Translation in progress.
Row 149: Translation done.
Row 150: Translation in progress.
Row 150: Translation done.
Row 151: Translation in progress.
Row 151: Translation done.
Row 152: Translation in progress.
Row 152: Translation done.
Row 153: Translation in progress.
Row 153: Translation done.
Row 154: Translation in progress.
Row 154: Translation done.
Row 155: Translation in progress.
Row 155: Translation don

Row 274: Translation done.
Row 275: Translation in progress.
Row 275: Translation done.
Row 276: Translation in progress.
Row 276: Translation done.
Row 277: Translation in progress.
Row 277: Translation done.
Row 278: Translation in progress.
Row 278: Translation done.
Row 279: Translation in progress.
Row 279: Translation done.
Row 280: Translation in progress.
Row 280: Translation done.
Row 281: Translation in progress.
Row 281: Translation done.
Row 282: Translation in progress.
Row 282: Translation done.
Row 283: Translation in progress.
Row 283: Translation done.
Row 284: Translation in progress.
Row 284: Translation done.
Row 285: Translation in progress.
Row 285: Translation done.
Row 286: Translation in progress.
Row 286: Translation done.
Row 287: Translation in progress.
Row 287: Translation done.
Row 288: Translation in progress.
Row 288: Translation done.
Row 289: Translation in progress.
Row 289: Translation done.
Row 290: Translation in progress.
Row 290: Translation don

Row 409: Translation done.
Row 410: Translation in progress.
Row 410: Translation done.
Row 411: Translation in progress.
Row 411: Translation done.
Row 412: Translation in progress.
Row 412: Translation done.
Row 413: Translation in progress.
Row 413: Translation done.
Row 414: Translation in progress.
Row 414: Translation done.
Row 415: Translation in progress.
Row 415: Translation done.
Row 416: Translation in progress.
Row 416: Translation done.
Row 417: Translation in progress.
Row 417: Translation done.
Row 418: Translation in progress.
Row 418: Translation done.
Row 419: Translation in progress.
Row 419: Translation done.
Row 420: Translation in progress.
Row 420: Translation done.
Row 421: Translation in progress.
Row 421: Translation done.
Row 422: Translation in progress.
Row 422: Translation done.
Row 423: Translation in progress.
Row 423: Translation done.
Row 424: Translation in progress.
Row 424: Translation done.
Row 425: Translation in progress.
Row 425: Translation don

Row 544: Translation done.
Row 545: Translation in progress.
Row 545: Translation done.
Row 546: Translation in progress.
Row 546: Translation done.
Row 547: Translation in progress.
Row 547: Translation done.
Row 548: Translation in progress.
Row 548: Translation done.
Row 549: Translation in progress.
Row 549: Translation done.
Row 550: Translation in progress.
Row 550: Translation done.
Row 551: Translation in progress.
Row 551: Translation done.
Row 552: Translation in progress.
Row 552: Translation done.
Row 553: Translation in progress.
Row 553: Translation done.
Row 554: Translation in progress.
Row 554: Translation done.
Row 555: Translation in progress.
Row 555: Translation done.
Row 556: Translation in progress.
Row 556: Translation done.
Row 557: Translation in progress.
Row 557: Translation done.
Row 558: Translation in progress.
Row 558: Translation done.
Row 559: Translation in progress.
Row 559: Translation done.
Row 560: Translation in progress.
Row 560: Translation don

Row 679: Translation done.
Row 680: Translation in progress.
Row 680: Translation done.
Row 681: Translation in progress.
Row 681: Translation done.
Row 682: Translation in progress.
Row 682: Translation done.
Row 683: Translation in progress.


In [9]:
df_jobs['Language'].value_counts()

en    1
Name: Language, dtype: int64

In [ ]:
if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl')
    
    df_jobs.to_csv(f'{data_dir}df_jobs_raw_language_detected.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')

In [ ]:
# Drop non-english job descriptions
# df_jobs.drop(df_jobs.index[df_jobs['Language'] == str(language)], axis='index', inplace=True, errors='ignore')


In [ ]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_language_detected.pkl').reset_index(drop=True)


In [ ]:
# Find and count unique search keywords
search_keywords = list(set(df_jobs['Search Keyword'].to_list()))


In [ ]:
len(search_keywords)


In [ ]:
search_keywords
